Current exercise has the following steps:
1. create a process that will discriminate computer code and identify a language by the code features.
2. the input training and test files needs to be prepared (presumed structure is a set of tuples [key feature, [set of features]).
3. the discriminating features of code are not yet defined. 
4. the files need to be processed for input.
5. Need to decide which classifier is useful.

Strategy and Initial observations:
1. Envisioning a set of data that includes a row number, a profile of regex distributions, and the code type. 
2. Need to determine how to read in a file name, capture the file extension and associate it with the regex text profile.
3. Immerse self in the TextBlob software. 
4. Need to understand the role of glob.glob.
